<a href="https://colab.research.google.com/github/H-Ismael/pubhealth/blob/main/models_code/enriched_imbalanced_data_roberta_lr0001_main_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BERT with Balanced and Enriched data**

In [1]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 33.7 MB/s eta 0:00:00


In [2]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 16.9 MB/s eta 0:00:00


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.8 MB/s eta 0:00:00


In [4]:
import pandas as pd 
from matplotlib import pyplot as plt

import torch 
from torch import nn, optim 
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from torchmetrics.functional import accuracy , f1_score

import pytorch_lightning as pl 
from pytorch_lightning.callbacks import TQDMProgressBar

import transformers 
from transformers import BertModel, BertConfig, AutoModel, BertTokenizerFast , RobertaModel , RobertaConfig , RobertaTokenizerFast

In [11]:
pub_health_train = pd.read_csv('drive/MyDrive/pubhealth/data/clean/exp1/clean_train.tsv', sep='\t')
pub_health_test  = pd.read_csv('drive/MyDrive/pubhealth/data/clean/exp1/clean_test.tsv', sep='\t')

In [ ]:
pub_health_train.describe()

,label,claim,main_text,explanation,subjects,sources
count,7727,7727,7727,7727,7727,7727
unique,4,7727,7586,7644,4709,4193
top,true,Health officials urge flu shots as virus becom...,Beta-Propiolactone is in vaccines and is known...,Environmental activist Greta Thunberg’s discou...,Health News,
freq,3001,1,9,2,749,3406


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pub_health_train.head()

,label,claim,main_text,explanation,subjects,sources
0,false,"""The money the Clinton Foundation took from fr...","""Hillary Clinton is in the political crosshair...","""Gingrich said the Clinton Foundation """"took m...","Foreign Policy, PunditFact, Newt Gingrich,",https://www.wsj.com/articles/clinton-foundatio...
1,mixture,Annual Mammograms May Have More False-Positives,While the financial costs of screening mammogr...,This article reports on the results of a study...,"Screening,WebMD,women's health",
2,mixture,SBRT Offers Prostate Cancer Patients High Canc...,The news release quotes lead researcher Robert...,This news release describes five-year outcomes...,"Association/Society news release,Cancer",https://www.healthnewsreview.org/wp-content/up...
3,true,"Study: Vaccine for Breast, Ovarian Cancer Has ...","The story does discuss costs, but the framing ...","While the story does many things well, the ove...","Cancer,WebMD,women's health",http://clinicaltrials.gov/ct2/results?term=can...
4,true,Some appendicitis cases may not require ’emerg...,"""Although the story didn’t cite the cost of ap...",We really don’t understand why only a handful ...,,


In [ ]:
def tolist(col):
  '''column of string to list
  '''
  return col.to_list()

def avglist(strlist:list):
  '''get average of a list of strings
  '''
  return sum([len(s) for s in strlist])/len(strlist)

def average_columns_len(df,columns:list):
  ''' given a list of columns in data frame , the function should return s dict having as values an average of the lenght of sentence present in the columns which are keys
  '''
  res = dict()
  for col in columns:
    res[col]=round(avglist(tolist(df[col])),2)
  return res


In [ ]:
average_columns_len(pub_health_train,['claim','main_text','subjects','explanation','sources'])

{'claim': 85.69,
 'main_text': 4376.84,
 'subjects': 40.08,
 'explanation': 482.97,
 'sources': 390.47}

In [ ]:
# l = 0
# bigs = ''
# pbhealth = pub_health_train['sources'].to_list()
# for s in pbhealth:
#     s = str(s)
#     if len(s) > l:
#       l = len(s)
#       bigs = s


In [13]:
pub_health_train['subjects'][0]

'Foreign Policy, PunditFact, Newt Gingrich, '

In [ ]:
pub_health_train['claim'][0]

'"The money the Clinton Foundation took from from foreign governments while Hillary Clinton was secretary of state ""is clearly illegal. … The Constitution says you can’t take this stuff."'

In [ ]:
pub_health_train['explanation'][0]

'"Gingrich said the Clinton Foundation ""took money from from foreign governments while (Hillary Clinton) was secretary of state. It is clearly illegal. … The Constitution says you can’t take this stuff."" A clause in the Constitution does prohibit U.S. officials such as former Secretary of State Hillary Clinton from receiving gifts, or emoluments, from foreign governments. But the gifts in this case were donations from foreign governments that went to the Clinton Foundation, not Hillary Clinton. She was not part of the foundation her husband founded while she was secretary of state. Does that violate the Constitution? Some libertarian-minded constitutional law experts say it very well could. Others are skeptical. What’s clear is there is room for ambiguity, and the donations are anything but ""clearly illegal."" The reality is this a hazy part of U.S. constitutional\xa0law. '

In [ ]:
pub_health_train['main_text'][0]

'"Hillary Clinton is in the political crosshairs as the author of a new book alleges improper financial ties between her public and personal life. At issue in conservative author Peter Schweizer’s forthcoming book Clinton Cash are donations from foreign governments to the Clinton Foundation during the four years she served as secretary of state. George Stephanopoulos used an interview with Schweizer on ABC This Week to point out what other nonpartisan journalists have found: There is no ""smoking gun"" showing that donations to the foundation influenced her foreign policy decisions. Still, former Republican House Speaker Newt Gingrich says the donations are ""clearly illegal"" under federal law. In his view, a donation by a foreign government to the Clinton Foundation while Clinton was secretary of state is the same as money sent directly to her, he said, even though she did not join the foundation’s board until she left her post. ""The Constitution of the United States says you cannot

In [16]:
# pub_health_train = pub_health_train[['claim','explanation','subject','main_text', 'label']]
# pub_health_train = pub_health_train[['claim','explanation','subject','main_text', 'label']]
pub_health_train = pub_health_train[['explanation','subjects','label']]
pub_health_test = pub_health_test[['explanation','subjects','label']]

In [17]:
pub_health_train['main_text_subj'] = 'Subject : ' + pub_health_train['subjects'] + '.' + 'Main : ' + pub_health_train['main_text']
pub_health_test['main_text_subj'] = 'Subject : ' + pub_health_test['subjects'] + '.' + 'Main : ' + pub_health_test['main_text']

In [18]:
pub_health_train['label'] = pub_health_train['label'].map(
    {"true":0, "false":1, "unproven":2, "mixture":3}
)
pub_health_test['label'] = pub_health_test['label'].map(
    {"true":0, "false":1, "unproven":2, "mixture":3}
)

In [24]:
class HealthClaimClassifier(pl.LightningModule):

    def __init__(self, max_seq_len=512, batch_size=8, learning_rate=1e-4):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        self.loss = nn.CrossEntropyLoss()

        self.pretrain_model = AutoModel.from_pretrained('roberta-base', return_dict=False)
        self.pretrain_model.eval() 
        for param in self.pretrain_model.parameters():
            param.requires_grad = False
        self.train_metric_list = []
        self.test_metric_list = []

        self.new_layers = nn.Sequential(
            nn.Linear(768, 512), 
            nn.ReLU(),
            nn.Dropout(0.2), 
            nn.Linear(512, 4),
            nn.LogSoftmax(dim=1),
        )
    def prepare_data(self):
        tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', return_dict=False)
        
        tokens_train = tokenizer.batch_encode_plus(
            pub_health_train['main_text_subj'].tolist(),
            max_length=self.max_seq_len, 
            pad_to_max_length=True, 
            truncation=True, 
            return_token_type_ids=False
        )

        tokens_test = tokenizer.batch_encode_plus(            
            pub_health_test['main_text_subj'].tolist(),
            max_length=self.max_seq_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=False
        )

        self.train_seq = torch.tensor(tokens_train['input_ids'])
        self.train_mask = torch.tensor(tokens_train['attention_mask'])
        self.train_y = torch.tensor(pub_health_train['label'].tolist())

        self.test_seq = torch.tensor(tokens_test['input_ids'])
        self.test_mask = torch.tensor(tokens_test['attention_mask'])
        self.test_y = torch.tensor(pub_health_test['label'].tolist())

    def forward(self, seq, mask):
        _, output = self.pretrain_model(seq, attention_mask=mask)
        output = self.new_layers(output)
        return output
    
    def train_dataloader(self):
        train_dataset = TensorDataset(self.train_seq, self.train_mask, self.train_y)
        self.train_dataloader_obj = DataLoader(train_dataset, batch_size=self.batch_size)
        return self.train_dataloader_obj
    
    def test_dataloader(self):
        test_dataset = TensorDataset(self.test_seq, self.test_mask, self.test_y)
        self.test_dataloader_obj = DataLoader(test_dataset, batch_size=self.batch_size)
        return self.test_dataloader_obj
    
    def training_step(self, batch, batch_idx):
        seq, mask, targets = batch 

        outputs = self(seq, mask)
        preds = torch.argmax(outputs, dim=1)
        
        train_accuracy = accuracy(preds, targets,task="multiclass", num_classes=4)
        train_f1core = f1_score(preds, targets,task="multiclass", num_classes=4)

        loss = self.loss(outputs, targets)

        self.log('train_accuracy', train_accuracy, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_f1core', train_f1core, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        metric = {'loss': loss, 'train_accuracy': train_accuracy , 'train_f1core':train_f1core}
        print(metric)
        self.train_metric_list.append(metric)
        return metric
    
    def test_step(self, batch, batch_idx):
        seq, mask, targets = batch 

        outputs = self.forward(seq, mask)
        preds = torch.argmax(outputs, dim=1)

        test_accuracy = accuracy(preds, targets,task="multiclass", num_classes=4)
        test_f1core = f1_score(preds, targets,task="multiclass", num_classes=4)
        loss = self.loss(outputs, targets)
        metric = {'loss': loss, 'test_accuracy': test_accuracy,'test_f1core':test_f1core}
        print(metric)
        self.test_metric_list.append(metric)
        return metric

    def test_epoch_end(self, outputs):
        test_outs = []
        for test_out in outputs:
            out = test_out['test_accuracy']
            test_outs.append(out)
        total_test_accuracy = torch.stack(test_outs).mean()
        self.log('total_test_accuracy', total_test_accuracy, on_step=False, on_epoch=True)
        return total_test_accuracy
    
    def configure_optimizers(self):
        params = self.parameters()
        optimizer = torch.optim.Adam(params=params, lr=self.learning_rate)
        return optimizer

In [ ]:
# model = HealthClaimClassifier()
# trainer = pl.Trainer(fast_dev_run=True, accelerator='gpu', devices=1)


In [25]:
model = HealthClaimClassifier()
bar = TQDMProgressBar()
# trainer = pl.Trainer(default_root_dir='BERT_transfer_learning_ckpts', max_epochs=10, accelerator='gpu', devices=1)
trainer = pl.Trainer(default_root_dir='roBERTa_transfer_learning_ckpts', max_epochs=12 , accelerator='gpu', devices=1)

histo = trainer.fit(model)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU avail

Training: 0it [00:00, ?it/s]

Streaming output truncated to the last 5000 lines.
{'loss': tensor(0.5152, device='cuda:0', grad_fn=<NllLossBackward0>), 'train_accuracy': tensor(1., device='cuda:0'), 'train_f1core': tensor(1., device='cuda:0')}
{'loss': tensor(0.8703, device='cuda:0', grad_fn=<NllLossBackward0>), 'train_accuracy': tensor(0.6250, device='cuda:0'), 'train_f1core': tensor(0.6250, device='cuda:0')}
{'loss': tensor(0.7158, device='cuda:0', grad_fn=<NllLossBackward0>), 'train_accuracy': tensor(0.6250, device='cuda:0'), 'train_f1core': tensor(0.6250, device='cuda:0')}
{'loss': tensor(0.8555, device='cuda:0', grad_fn=<NllLossBackward0>), 'train_accuracy': tensor(0.6250, device='cuda:0'), 'train_f1core': tensor(0.6250, device='cuda:0')}
{'loss': tensor(0.7757, device='cuda:0', grad_fn=<NllLossBackward0>), 'train_accuracy': tensor(0.5000, device='cuda:0'), 'train_f1core': tensor(0.5000, device='cuda:0')}
{'loss': tensor(0.3243, device='cuda:0', grad_fn=<NllLossBackward0>), 'train_accuracy': tensor(1., device='

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=12` reached.


In [ ]:
# model = HealthClaimClassifier()
# bar = TQDMProgressBar()
# trainer = pl.Trainer(default_root_dir='BERT_transfer_learning_ckpts', max_epochs=10, accelerator='gpu', devices=1, callbacks=[bar])
# histo = trainer.fit(model)

In [26]:
trainer.test()

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at roBERTa_transfer_learning_ckpts/lightning_logs/version_0/checkpoints/epoch=11-step=14712.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at roBERTa_transfer_learning_ckpts/lightning_logs/version_0/checkpoints/epoch=11-step=14712.ckpt


Testing: 0it [00:00, ?it/s]

{'loss': tensor(1.2563, device='cuda:0'), 'test_accuracy': tensor(0.3750, device='cuda:0'), 'test_f1core': tensor(0.3750, device='cuda:0')}
{'loss': tensor(1.3483, device='cuda:0'), 'test_accuracy': tensor(0.6250, device='cuda:0'), 'test_f1core': tensor(0.6250, device='cuda:0')}
{'loss': tensor(1.0008, device='cuda:0'), 'test_accuracy': tensor(0.6250, device='cuda:0'), 'test_f1core': tensor(0.6250, device='cuda:0')}
{'loss': tensor(0.7186, device='cuda:0'), 'test_accuracy': tensor(0.7500, device='cuda:0'), 'test_f1core': tensor(0.7500, device='cuda:0')}
{'loss': tensor(0.5938, device='cuda:0'), 'test_accuracy': tensor(0.7500, device='cuda:0'), 'test_f1core': tensor(0.7500, device='cuda:0')}
{'loss': tensor(0.6820, device='cuda:0'), 'test_accuracy': tensor(0.7500, device='cuda:0'), 'test_f1core': tensor(0.7500, device='cuda:0')}
{'loss': tensor(0.5526, device='cuda:0'), 'test_accuracy': tensor(0.8750, device='cuda:0'), 'test_f1core': tensor(0.8750, device='cuda:0')}
{'loss': tensor(1.34

[{'total_test_accuracy': 0.6072580814361572}]